In [ ]:
import elasticsearch
from elasticsearch import Elasticsearch

In [ ]:
es = Elasticsearch([{'host':'localhost','port':9200}])

In [ ]:
if es.ping():
    print('Connected') 

Connected


/home/hitesh/Desktop/pyod_project/tasl/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [ ]:
#collecting yesterday log data and use it for training
from datetime import datetime,timedelta
yesterday = datetime.now()-timedelta(1)
today =datetime.today().strftime('%Y.%m.%d')

In [ ]:
import pandas as pd
import numpy as np

# Import models
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
# from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy import stats
from numpy import percentile
from joblib import dump, load

In [ ]:
import json
from elasticsearch.helpers import scan
rest = scan(es,index="logstash-"+(datetime.strftime(yesterday,'%Y.%m.%d')),query={"query":{"match_all" : {}}})
train_log=list()
for item in rest:
    if item!=0:
        train_log.append(item["_source"]["message"])
    else:
        pas
    #print(op['_source']['message'])

/home/hitesh/Desktop/pyod_project/tasl/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [ ]:
import pycef
parsed_train_log=list()
for i in train_log:
    op = pycef.parse(i)
    parsed_train_log.append(op)

In [ ]:
train_df_ini = pd.DataFrame(parsed_train_log)
train_df_ini.to_csv("train.csv")

In [ ]:
print(type(train_df_ini))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
def data_preproc (tempdf):
    tempdf = tempdf.apply(le.fit_transform)
    return tempdf

In [ ]:
dft = data_preproc(train_df_ini)

In [ ]:
dft

,DeviceVendor,DeviceProduct,DeviceVersion,DeviceEventClassID,Name,DeviceName,Severity,DeviceSeverity,CEFVersion,eventId,...,ad.HomePath,ad.UserPrincipalName,ad.AllowedToDelegateTo,ad.UserWorkstations,Account Domain and Name,ad.Dummy,Old User Account Control Value,New User Account Control Value,Changes in User Account Control,Reason or Error Code
0,1,1,1,16,4,4,1,1,0,69,...,1,1,1,1,3,1,1,1,1,1
1,1,1,1,16,4,4,1,1,0,70,...,1,1,1,1,3,1,1,1,1,1
2,1,1,1,16,4,4,1,1,0,71,...,1,1,1,1,3,1,1,1,1,1
3,1,1,1,16,4,4,1,1,0,72,...,1,1,1,1,3,1,1,1,1,1
4,1,1,1,16,4,4,1,1,0,73,...,1,1,1,1,3,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96443,1,1,2,13,12,12,1,1,0,96439,...,1,1,1,1,3,1,1,1,1,1
96444,1,1,2,18,34,34,1,1,0,96440,...,1,1,1,1,3,1,1,1,1,1
96445,1,1,2,47,32,32,1,1,0,96445,...,1,1,1,1,3,1,1,1,1,1
96446,1,1,2,48,36,36,1,1,0,96446,...,1,1,1,1,3,1,1,1,1,1


In [ ]:
def if_train(tempdf):
  scaler=StandardScaler()
  scaler.fit(tempdf)
  scaled_data=scaler.transform(tempdf)
  pca=PCA(n_components=2)
  pca.fit(scaled_data)
  x_pca = pca.transform(scaled_data)
  tempdf = pd.DataFrame(data = x_pca, index = list(range(0,len(x_pca[:,0]))), columns = ["prop1","prop2"])
  X1 = tempdf['prop1'].values.reshape(-1,1)
  X2 = tempdf['prop2'].values.reshape(-1,1)
  X = np.concatenate((X1,X2),axis=1)
  clf =IForest(contamination=0.05,random_state=0)
  clf.fit(X)
  dump(clf, 'if.joblib')

In [ ]:
if_train(dft)

In [ ]:
es.indices.refresh(index="logstash-"+(datetime.today().strftime('%Y.%m.%d')))
es.count(index="logstash-"+(datetime.today().strftime('%Y.%m.%d')),body='{"query":{"match_all":{}}}')

/home/hitesh/Desktop/pyod_project/tasl/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'count': 34015,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [ ]:
yes_count = es.count(index="logstash-"+(datetime.today().strftime('%Y.%m.%d')),body='{"query":{"match_all":{}}}')['count']

In [ ]:
print(yes_count)

34015


In [ ]:
import json
from elasticsearch.helpers import scan

res = scan(es,index="logstash-"+(datetime.today().strftime('%Y.%m.%d')),query={"query":{"match_all" : {}}})
input_log=list()
for item in res:
    if item!=0:
        input_log.append(item["_source"]["message"])
    else:
        pass
    #print(op['_source']['message'])

In [ ]:
import pandas as pd
inp_df = pd.DataFrame(input_log)

In [ ]:
print(type(inp_df))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
import pycef
parsed_inp_log=list()
for i in input_log:
    op = pycef.parse(i)
    parsed_inp_log.append(op)

In [ ]:
print(len(parsed_inp_log))

34015


In [ ]:
inp = pd.DataFrame(parsed_inp_log)
inp.to_csv("input.csv")

In [ ]:
print(len(inp))

34015


In [ ]:
dfi = data_preproc(inp)

In [ ]:
print(dfi.shape)

(34015, 170)


In [ ]:
dfi.head()

,DeviceVendor,DeviceProduct,DeviceVersion,DeviceEventClassID,Name,DeviceName,Severity,DeviceSeverity,CEFVersion,eventId,...,Old User Account Control Value,New User Account Control Value,Changes in User Account Control,ad.OldUacValue,ad.DnsHostName,ad.NewUacValue,ad.UserAccountControl,ad.ServicePrincipalNames,ad.ComputerAccountChange,Account Domain and Name
0,1,1,2,16,4,4,1,1,0,0,...,1,1,1,3,1,3,3,1,1,2
1,1,1,2,14,11,11,3,3,0,1,...,1,1,1,3,1,3,3,1,1,2
2,1,1,2,15,12,12,1,1,0,2,...,1,1,1,3,1,3,3,1,1,2
3,1,1,2,16,4,4,1,1,0,3,...,1,1,1,3,1,3,3,1,1,2
4,1,1,2,14,11,11,3,3,0,4,...,1,1,1,3,1,3,3,1,1,2


In [ ]:
def IF(tempdf,ini_tempdf):
    global df1
    temp = tempdf
    scaler=StandardScaler()
    scaler.fit(temp)
    scaled_data=scaler.transform(temp)
    pca=PCA(n_components=2)
    pca.fit(scaled_data)
    x_pca = pca.transform(scaled_data)
    temp = pd.DataFrame(data = x_pca, index = list(range(0,len(x_pca[:,0]))), columns = ["prop1","prop2"])
    X1 = temp['prop1'].values.reshape(-1,1)
    X2 = temp['prop2'].values.reshape(-1,1)
    X = np.concatenate((X1,X2),axis=1)
    print(len(X1))
    print(len(X2))
    outliers_fraction = 0.05
    xx , yy = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
  
    obj = load('if.joblib')
    y_pred = obj.predict(X)
    scores_pred = obj.decision_function(X) * -1
  
    n_inliers = len(y_pred) - np.count_nonzero(y_pred)
    n_outliers = np.count_nonzero(y_pred == 1)
    plt.figure(figsize=(8, 8))
  # copy of dataframe
    df1 = ini_tempdf
    outlier_list = y_pred.tolist()
    df1['Outlier']=outlier_list
    df1['LineId'] = [x for x in range(len(outlier_list)) ]
    
  # sales - inlier feature 1,  profit - inlier feature 2
    inliers_sales = np.array(temp['prop1'][df1['Outlier'] == 0]).reshape(-1,1)
    inliers_profit = np.array(temp['prop2'][df1['Outlier'] == 0]).reshape(-1,1)
    
  # sales - outlier feature 1, profit - outlier feature 2
    outliers_sales = temp['prop1'][df1['Outlier'] == 1].values.reshape(-1,1)
    outliers_profit = temp['prop2'][df1['Outlier'] == 1].values.reshape(-1,1)
         
    print('OUTLIERS:',n_outliers,'INLIERS:',n_inliers)
#    return ini_tempdf

In [ ]:
df1 = pd.DataFrame()

In [ ]:
IF(dfi,inp_df)

34015
34015
OUTLIERS: 6583 INLIERS: 27432


<Figure size 576x576 with 0 Axes>

In [ ]:
df1

,0,Outlier,LineId
0,CEF:0|Microsoft|Microsoft Windows|Windows Serv...,0,0
1,CEF:0|Microsoft|Microsoft Windows|Windows Serv...,1,1
2,CEF:0|Microsoft|Microsoft Windows|Windows Serv...,0,2
3,CEF:0|Microsoft|Microsoft Windows|Windows Serv...,0,3
4,CEF:0|Microsoft|Microsoft Windows|Windows Serv...,1,4
...,...,...,...
34010,CEF:0|Microsoft|Microsoft Windows|Windows Serv...,0,34010
34011,CEF:0|Microsoft|Microsoft Windows|Windows Serv...,0,34011
34012,CEF:0|Microsoft|System or Application Event|Wi...,0,34012
34013,CEF:0|Microsoft|Microsoft Windows|Windows Serv...,0,34013


In [ ]:
df1_dict = df1.to_dict(orient="records")

In [ ]:
if es.indices.exists(index="output_cef-log-index")==True:
    es.indices.delete(index="output_cef-log-index")
else:
    pass

/home/hitesh/Desktop/pyod_project/tasl/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [ ]:
if es.indices.exists(index="output_cef-log-index")==True:
    count_op = es.count(index="output_cef-log-index",body='{"query":{"match_all":{}}}')['count']
    if count_op!=0:
        for i in range(1,count+1):
            es.delete(index="output_cef-log-index",id=i)
else:
    es.indices.create(index="output_cef-log-index")

In [ ]:
for i in range(1,(len(df1_dict)+1)):
    es.index(index="output_cef-log-index",id=i,body=df1_dict[(i-1)])

In [ ]:
es.indices.refresh(index="output_cef-log-index")

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [ ]:
count_op = es.count(index="output_cef-log-index",body='{"query":{"match_all":{}}}')['count']

In [ ]:
count_op

34015